In [1]:
import xarray as xr
import geopandas as gpd
import glob

In [2]:
ug = gpd.read_file(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/uganda_administrative_boundaries/simplified_uganda_districts.shp"
)

ug.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [3]:
ug.head()

,ADM2_EN,ADM1_EN,geometry
0,Abim,Northern,"POLYGON ((33.58099 3.14912, 33.58555 3.14702, ..."
1,Adjumani,Northern,"POLYGON ((32.04597 3.58896, 32.04578 3.58875, ..."
2,Agago,Northern,"POLYGON ((33.41652 3.30434, 33.41701 3.30403, ..."
3,Alebtong,Northern,"POLYGON ((33.03629 2.5013, 33.03664 2.50075, 3..."
4,Amolatar,Northern,"POLYGON ((32.9014 1.8057, 32.90231 1.80568, 32..."


In [4]:
ds = xr.open_dataset(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/era5/pressure/surfacepressure1981-1987.grib",
    engine="cfgrib",
)

ds

<xarray.Dataset> Size: 2GB
Dimensions:     (time: 61344, latitude: 24, longitude: 259)
Coordinates:
  * time        (time) datetime64[ns] 491kB 1981-01-01 ... 1987-12-31T23:00:00
  * latitude    (latitude) float64 192B 4.25 4.0 3.75 3.5 ... -1.0 -1.25 -1.5
  * longitude   (longitude) float64 2kB -29.5 -29.25 -29.0 ... 34.5 34.75 35.0
    number      int64 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
    valid_time  (time) datetime64[ns] 491kB ...
Data variables:
    sp          (time, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T12:16 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
minx, miny, maxx, maxy = ug.total_bounds

ds_ug = ds.sel(longitude=slice(minx, maxx), latitude=slice(maxy, miny))
ds_ug

<xarray.Dataset> Size: 120MB
Dimensions:     (time: 61344, latitude: 22, longitude: 22)
Coordinates:
  * time        (time) datetime64[ns] 491kB 1981-01-01 ... 1987-12-31T23:00:00
  * latitude    (latitude) float64 176B 4.0 3.75 3.5 3.25 ... -0.75 -1.0 -1.25
  * longitude   (longitude) float64 176B 29.75 30.0 30.25 ... 34.5 34.75 35.0
    number      int64 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
    valid_time  (time) datetime64[ns] 491kB ...
Data variables:
    sp          (time, latitude, longitude) float32 119MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T12:16 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
ds_monthly = ds_ug.resample(time="1M").mean()
ds_monthly

c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


<xarray.Dataset> Size: 164kB
Dimensions:    (time: 84, latitude: 22, longitude: 22)
Coordinates:
  * time       (time) datetime64[ns] 672B 1981-01-31 1981-02-28 ... 1987-12-31
  * latitude   (latitude) float64 176B 4.0 3.75 3.5 3.25 ... -0.75 -1.0 -1.25
  * longitude  (longitude) float64 176B 29.75 30.0 30.25 ... 34.5 34.75 35.0
    number     int64 8B 0
    step       timedelta64[ns] 8B 00:00:00
    surface    float64 8B 0.0
Data variables:
    sp         (time, latitude, longitude) float32 163kB 9.195e+04 ... 8.307e+04
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T12:16 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
files = glob.glob(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/era5/pressure/*.grib"
)

datasets = []

for file in files:
    ds = xr.open_dataset(file, engine="cfgrib")
    minx, miny, maxx, maxy = ug.total_bounds
    ds_ug = ds.sel(longitude=slice(minx, maxx), latitude=slice(maxy, miny))
    ds_monthly = ds_ug.resample(time="1M").mean()
    datasets.append(ds_monthly)

combined_ds = xr.concat(datasets, dim="time")

combined_ds

Ignoring index file 'C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/era5/pressure\\surfacepressure1981-1987.grib.5b7b6.idx' incompatible with GRIB file
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Program Files\Python

<xarray.Dataset> Size: 1MB
Dimensions:    (time: 539, latitude: 22, longitude: 22)
Coordinates:
  * time       (time) datetime64[ns] 4kB 1981-01-31 1981-02-28 ... 2025-11-30
  * latitude   (latitude) float64 176B 4.0 3.75 3.5 3.25 ... -0.75 -1.0 -1.25
  * longitude  (longitude) float64 176B 29.75 30.0 30.25 ... 34.5 34.75 35.0
    number     int64 8B 0
    step       timedelta64[ns] 8B 00:00:00
    surface    float64 8B 0.0
Data variables:
    sp         (time, latitude, longitude) float32 1MB 9.195e+04 ... 8.319e+04
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T12:19 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
df_pressure = combined_ds.to_dataframe().reset_index()
df_pressure

,time,latitude,longitude,sp,number,step,surface
0,1981-01-31,4.00,29.75,91954.375000,0,0 days,0.0
1,1981-01-31,4.00,30.00,91553.093750,0,0 days,0.0
2,1981-01-31,4.00,30.25,91427.914062,0,0 days,0.0
3,1981-01-31,4.00,30.50,90955.250000,0,0 days,0.0
4,1981-01-31,4.00,30.75,90770.023438,0,0 days,0.0
...,...,...,...,...,...,...,...
260871,2025-11-30,-1.25,34.00,88258.765625,0,0 days,0.0
260872,2025-11-30,-1.25,34.25,87118.000000,0,0 days,0.0
260873,2025-11-30,-1.25,34.50,85187.546875,0,0 days,0.0
260874,2025-11-30,-1.25,34.75,83675.171875,0,0 days,0.0


In [ ]:
# convert grid cells points to shapely points
gdf = gpd.GeoDataFrame(
    df_pressure,
    geometry=gpd.points_from_xy(df_pressure.longitude, df_pressure.latitude),
    crs="EPSG:4326",
)

In [10]:
gdf = gpd.sjoin(gdf, ug, how="left", predicate="within")
gdf

,time,latitude,longitude,sp,number,step,surface,geometry,index_right,ADM2_EN,ADM1_EN
0,1981-01-31,4.00,29.75,91954.375000,0,0 days,0.0,POINT (29.75 4),NaN,NaN,NaN
1,1981-01-31,4.00,30.00,91553.093750,0,0 days,0.0,POINT (30 4),NaN,NaN,NaN
2,1981-01-31,4.00,30.25,91427.914062,0,0 days,0.0,POINT (30.25 4),NaN,NaN,NaN
3,1981-01-31,4.00,30.50,90955.250000,0,0 days,0.0,POINT (30.5 4),NaN,NaN,NaN
4,1981-01-31,4.00,30.75,90770.023438,0,0 days,0.0,POINT (30.75 4),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
260871,2025-11-30,-1.25,34.00,88258.765625,0,0 days,0.0,POINT (34 -1.25),NaN,NaN,NaN
260872,2025-11-30,-1.25,34.25,87118.000000,0,0 days,0.0,POINT (34.25 -1.25),NaN,NaN,NaN
260873,2025-11-30,-1.25,34.50,85187.546875,0,0 days,0.0,POINT (34.5 -1.25),NaN,NaN,NaN
260874,2025-11-30,-1.25,34.75,83675.171875,0,0 days,0.0,POINT (34.75 -1.25),NaN,NaN,NaN


In [11]:
df_monthly = gdf.groupby(["ADM2_EN", "ADM1_EN", "time"])["sp"].mean().reset_index()
df_monthly

,ADM2_EN,ADM1_EN,time,sp
0,Abim,Northern,1981-01-31,88380.492188
1,Abim,Northern,1981-02-28,88324.414062
2,Abim,Northern,1981-03-31,88366.914062
3,Abim,Northern,1981-04-30,88423.492188
4,Abim,Northern,1981-05-31,88494.125000
...,...,...,...,...
63235,Zombo,Northern,2025-07-31,86811.125000
63236,Zombo,Northern,2025-08-31,86739.304688
63237,Zombo,Northern,2025-09-30,86690.109375
63238,Zombo,Northern,2025-10-31,86626.632812


In [12]:
# rename columns
df_monthly = df_monthly.rename(
    columns={
        "ADM2_EN": "District",
        "ADM1_EN": "Region",
        "time": "Date",
        "sp": "Surface_Pressure_Pa",
    }
)
df_monthly

,District,Region,Date,Surface_Pressure_Pa
0,Abim,Northern,1981-01-31,88380.492188
1,Abim,Northern,1981-02-28,88324.414062
2,Abim,Northern,1981-03-31,88366.914062
3,Abim,Northern,1981-04-30,88423.492188
4,Abim,Northern,1981-05-31,88494.125000
...,...,...,...,...
63235,Zombo,Northern,2025-07-31,86811.125000
63236,Zombo,Northern,2025-08-31,86739.304688
63237,Zombo,Northern,2025-09-30,86690.109375
63238,Zombo,Northern,2025-10-31,86626.632812


In [14]:
# save to csv
df_monthly.to_csv("uganda_monthly_pressure_1981_2025.csv", index=False)